# Package 

In [20]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from gensim.similarities.annoy import AnnoyIndexer

from utils import *

import os
import pickle
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
import polars as pl
from annoy import AnnoyIndex
import polars as pl
import polars as pl
from utils import *
from src.eval import model_eval
from src.config import (
    raw_data_session_id_dir, candidate_dir, model_for_eval, candidate_file_name, submit_file_name
    # , w2v_model_file_name, w2v_index_file_name
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [21]:
debug = False

topn = 100
model_version = 'w2v_v3'
if debug:
    n_rows = 100
else:
    n_rows = None

    
task = 'task1'
# Common setting
model_dir = f'../model_training/{model_version}/'
# target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']
submit_file = os.path.join('../data/sub_files/', 
                           submit_file_name.format(
                                task=task
                               , model_version=model_version
                               , model_for_eval=model_for_eval
                               , topn=topn
                           )
                          )

train_cg_file = os.path.join(base_dir,
                             candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='train'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
eval_cg_file = os.path.join(base_dir,
                            candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='eval'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
test_cg_file = os.path.join(base_dir,
                            candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='test'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )

# specific setting
num_tree = 100
# if model_for_eval:
w2v_model_file = os.path.join(model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(model_dir, f"{str(num_tree)}_{model_for_eval}.index")

# Load model 

In [76]:
w2vec = Word2Vec.load(w2v_model_dir)

In [26]:
test_sessions = read_test_data(task, test_data_dir=test_data_dir)
test_sessions.sample(5)

,prev_items,locale
135025,"[B09KCHLWH4, B09KCHLWH4, B09F5V3CDN, B08MC3BQK...",JP
184024,"[B08R71STV6, B0BJCXF6QK, B0B9V3B882, B08R71STV...",JP
309425,"[B0007VTA5S, B01N7W8FRS, B08S8RHJTN]",UK
313773,"[B09D8QR3HS, B09D8QR3HS, B07DD5YHMH, B07DC5PPF...",UK
258407,"[B097DNKW7L, B09YMB42RM, B097D5JQVH, B097D84HQW]",UK


In [39]:
test_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316971 entries, 0 to 316970
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   prev_items  316971 non-null  object
 1   locale      316971 non-null  object
dtypes: object(2)
memory usage: 4.8+ MB


In [27]:
# test_sessions['prev_items'] = test_sessions.apply(lambda row: process_item_lst(row), axis=1)

In [104]:
df = test_sessions.sample(1000)

In [105]:
for col in df.columns:
    df[col] = df[col].astype(str)

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 305876 to 237644
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prev_items  1000 non-null   object
 1   locale      1000 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


In [108]:
def get_rec(prev_items):
    # print(prev_items)
    res = [ele.replace('[', '').replace(']', '').replace('\n', '').replace("'", '').replace(' ', '') for ele in prev_items.split(' ')]
    
    try:
        similarity_dic = w2vec.wv.most_similar(positive=prev_items, topn=100)
        res = [item for item, simi in similarity_dic] 
        # print(err)
    except Exception as e:
        print(e)
    
    return res

pl_df = pl.from_dataframe(df)
pl_df = (
    pl_df
        .with_column(pl.col('prev_items').apply(lambda row: get_rec(row)).alias('next_item_prediction'))
)
result_df = pl_df.to_pandas()

In [109]:
# pl_df

In [112]:
result_df.head()

,prev_items,locale,next_item_prediction
0,"['B0BCSSGR43', 'B09ZYT2MQY']",UK,"[B0BCSSGR43,, B09ZYT2MQY]"
1,"['B08FC3QNBV', 'B09TW38KVZ', 'B0B1CZJZK8', 'B0...",UK,"[B08FC3QNBV,, B09TW38KVZ,, B0B1CZJZK8,, B09CTT..."
2,"['B0BLTWKKKG', 'B07SLHM55V', 'B014P55MN0']",JP,"[B0BLTWKKKG,, B07SLHM55V,, B014P55MN0]"
3,"['B07VDBFRZW', 'B08TQM4F53', 'B07VDBFRZW', 'B0...",JP,"[B07VDBFRZW,, B08TQM4F53,, B07VDBFRZW,, B07QCB..."
4,"['B00F5JMDHE', 'B08SJNJLPQ', 'B07W5Z3YVQ', 'B0...",JP,"[B00F5JMDHE,, B08SJNJLPQ,, B07W5Z3YVQ,, B07W5Z..."


# Polars pl.element() 

In [22]:
raw_data_session_id_dir

'data/raw_data_session_id'

In [23]:
! ls ../data/raw_data_session_id

product_unique2id.json	sessions_test_task1.parquet  sessions_train.parquet
products_train.parquet	sessions_test_task2.parquet
sessions_eval.parquet	sessions_test_task3.parquet


In [24]:
target_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'),
                          n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
file_path = '../model_training/als_v1/itemid2item.json'
with open(file_path, 'rb') as f:
    itemid2item = pickle.load(f)




In [25]:
item2itemid = {value: key for key, value in itemid2item.items()}

In [26]:
# item2itemid

In [27]:
target_pl.head().collect()

prev_items,next_item,locale,session_id
list[str],str,str,i64
"[""B09W9FND7K"", ""B09JSPLN1M""]","""B09M7GY217""","""DE""",0
"[""B076THCGSG"", ""B007MO8IME"", … ""B001B4TKA0""]","""B001B4THSA""","""DE""",1
"[""B0B1LGXWDS"", ""B00AZYORS2"", … ""B00AZYORS2""]","""B0767DTG2Q""","""DE""",2
"[""B0749V8TC7"", ""B0749W93VC"", ""B0749TX4YP""]","""B0749TX4YS""","""DE""",5
"[""B09SMK3R8H"", ""B01N4ND0F9""]","""B08YNZT93Z""","""DE""",6


In [28]:
target_pl.schema

{'prev_items': Unknown, 'next_item': Utf8, 'locale': Utf8, 'session_id': Int64}

In [29]:
# dir(pl.element())

In [30]:
# item2itemid['B09W9FND7K']

In [31]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [32]:
%%time
%memit

def map_item(lst):
    item_ids  = [item2itemid[item] for item in lst]
    return item_ids

df1 = (
    target_pl.with_columns(
                pl.col('prev_items').apply(lambda x: map_item(x)).alias('prev_item_ids')
    ).select(
        ['session_id', 'prev_items', 'prev_item_ids']
    )
).collect()
# print(df1.shape)
# print(df1.head(2))

peak memory: 1638.39 MiB, increment: 0.00 MiB
CPU times: user 1min 51s, sys: 4.16 s, total: 1min 55s
Wall time: 1min 56s


In [33]:
%%time
%memit

df2 = (
    target_pl.explode('prev_items')
        .with_columns(
                pl.col('prev_items').map_dict(item2itemid).alias('prev_item_ids'))
        .groupby('session_id')
        .agg(
            pl.col('prev_items')
            , pl.col('prev_item_ids')
        )
).collect()
print(df2.shape)
print(df2.head(2))

peak memory: 2194.60 MiB, increment: 0.01 MiB
(2946273, 3)
shape: (2, 3)
┌────────────┬──────────────────────────────┬──────────────────┐
│ session_id ┆ prev_items                   ┆ prev_item_ids    │
│ ---        ┆ ---                          ┆ ---              │
│ i64        ┆ list[str]                    ┆ list[i64]        │
╞════════════╪══════════════════════════════╪══════════════════╡
│ 1418688    ┆ ["B079STNCF9", "B07C91ZFQC"] ┆ [350046, 362613] │
│ 1942624    ┆ ["B07VKZLFVM", "B07VJLJWYN"] ┆ [497019, 496554] │
└────────────┴──────────────────────────────┴──────────────────┘
CPU times: user 50.1 s, sys: 3.01 s, total: 53.2 s
Wall time: 48.9 s


# W2V in details 

In [4]:
eval_pl = pl.scan_parquet('../data/candidates/task1_eval_w2v_v3_True_top100.parquet')
original_eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))


In [5]:
eval_pl.head().collect()

session_id,next_item_prediction
i64,list[str]
3,"[""B0B7HZ2GWX"", ""B09XMTWDVT"", … ""B09K3WNKZ4""]"
4,"[""B09Y5CSL3T"", ""B09Y5CW56M"", … ""B08LKS2KFV""]"
14,"[""B07LFQPX5L"", ""B07LFPJTVF"", … ""B09BJY1LBB""]"
15,"[""B086W985G3"", ""B07YQ2KSM4"", … ""B088FM4Y3B""]"
27,"[""B08XB9HWK4"", ""B09K3ZDNLH"", … ""B09R1TL35Z""]"


In [6]:
original_eval_pl.head().collect()

prev_items,next_item,locale,session_id
list[str],str,str,i64
"[""B09XMTWDVT"", ""B0B4MZZ8MB"", … ""B0B71CHT1L""]","""B0B4R9NN4B""","""DE""",3
"[""B09Y5CSL3T"", ""B09Y5DPTXN"", ""B09FKD61R8""]","""B0BGVBKWGZ""","""DE""",4
"[""B07LFQPX5L"", ""B07LFPJTVF""]","""B07LFRP5SS""","""DE""",14
"[""B085QWM3KB"", ""B001BAAV5W"", … ""B000VVMY48""]","""B000VVSWBM""","""DE""",15
"[""B01NAQR0Y2"", ""B08XBZL5CM""]","""B09ZPNHQ12""","""DE""",27


In [7]:
merged_pl = original_eval_pl.join(eval_pl, how='left', on='session_id' )

In [13]:
merged_pl.head().collect()

prev_items,next_item,locale,session_id,next_item_prediction
list[str],str,str,i64,list[str]
"[""B09XMTWDVT"", ""B0B4MZZ8MB"", … ""B0B71CHT1L""]","""B0B4R9NN4B""","""DE""",3,"[""B0B7HZ2GWX"", ""B09XMTWDVT"", … ""B09K3WNKZ4""]"
"[""B09Y5CSL3T"", ""B09Y5DPTXN"", ""B09FKD61R8""]","""B0BGVBKWGZ""","""DE""",4,"[""B09Y5CSL3T"", ""B09Y5CW56M"", … ""B08LKS2KFV""]"
"[""B07LFQPX5L"", ""B07LFPJTVF""]","""B07LFRP5SS""","""DE""",14,"[""B07LFQPX5L"", ""B07LFPJTVF"", … ""B09BJY1LBB""]"
"[""B085QWM3KB"", ""B001BAAV5W"", … ""B000VVMY48""]","""B000VVSWBM""","""DE""",15,"[""B086W985G3"", ""B07YQ2KSM4"", … ""B088FM4Y3B""]"
"[""B01NAQR0Y2"", ""B08XBZL5CM""]","""B09ZPNHQ12""","""DE""",27,"[""B08XB9HWK4"", ""B09K3ZDNLH"", … ""B09R1TL35Z""]"


In [12]:
# def get_next_items(x):
#     prev_items = x['prev_items']
#     local_rec = x['next_item_prediction']
#     final = [ele for ele in local_rec if ele in prev_items]
#     return len(final)

# (merged_pl
#     .with_columns(
        
#     pl.struct(["prev_items", "next_item_prediction"]).apply(
#                         lambda x: get_next_items(x)).alias('duplicate_num'))
#     # .select(pl.col('duplicate_num')).collect().describe()
# ).head().collect()
